# Cross Encoder Reranker

## 개요

Cross encoder reranker는 검색 증강 생성(RAG) 시스템의 성능을 향상시키기 위해 사용되는 기술입니다. 이 문서는 Hugging Face의 cross encoder 모델을 사용하여 retriever에서 reranker를 구현하는 방법을 설명합니다.

## 주요 특징 및 작동 방식

1. **목적**: 검색된 문서들의 순위를 재조정하여 질문에 가장 관련성 높은 문서를 상위로 올림
2. **구조**: 질문과 문서를 동시에 입력으로 받아 처리
3. **작동 방식**:
  - 질문과 문서를 하나의 입력으로 사용하여 유사도를 직접 출력
  - Self-attention 메커니즘을 통해 질문과 문서를 동시에 분석
4. **장점**:
  - 더 정확한 유사도 측정 가능
  - 질문과 문서 사이의 의미론적 유사성을 깊이 탐색
5. **한계점**:
  - 연산 비용이 높고 시간이 오래 걸림
  - 대규모 문서 집합에 직접 적용하기 어려움

## 실제 사용

- 일반적으로 초기 검색에서 상위 k개의 문서에 대해서만 reranking 수행
- Bi-encoder로 빠르게 후보를 추출한 후, Cross encoder로 정확도를 높이는 방식으로 활용

## 구현

- Hugging Face의 cross encoder 모델 또는 BAAI/bge-reranker와 같은 모델 사용
- LangChain 등의 프레임워크에서 CrossEncoderReranker 컴포넌트를 통해 쉽게 통합 가능

## Reranker의 주요 장점

1. 더 정확한 유사도 측정
2. 심층적인 의미론적 유사성 탐색
3. 검색 결과 개선
4. RAG 시스템 성능 향상
5. 유연한 통합
6. 다양한 사전 학습 모델 선택 가능

## Reranker 사용 시 문서 수 설정

- 일반적으로 상위 5~10개 문서에 대해 reranking 수행
- 최적의 문서 수는 실험과 평가를 통해 결정 필요

## Reranker 사용시 Trade-offs

1. 정확도 vs 처리 시간
2. 성능 향상 vs 계산 비용
3. 검색 속도 vs 관련성 정확도
4. 시스템 요구사항 충족
5. 데이터셋 특성 고려

간단한 예시를 통해 Cross Encoder Reranker의 구현 방법을 설명하겠습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
from langchain_chroma import Chroma

# API KEY 정보로드
load_dotenv()

True

In [2]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [3]:
from langchain.schema import Document

def get_sample_docs():
    return [
        Document(
            page_content="미국의 수도는 워싱턴이고, 일본은 도쿄이며 북한은 평양이다."
        ),
        Document(
            page_content="대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다."
        ),
        Document(
            page_content="서울은 대한민국의 수도로, 정치, 경제, 문화의 중심지입니다."
        ),
        Document(
            page_content="많은 사람들이 대구를 대한민국의 수도로 착각하지만, 실제 수도는 아닙니다."
        ),
        Document(page_content="한국의 수도는 서울이며, 세계적으로 유명한 도시입니다."),
        Document(page_content="대한민국의 가장 큰 도시는 인천이지만, 수도는 아닙니다."),
        Document(
            page_content="서울은 대한민국의 수도로서, 1948년부터 공식적으로 지정되었습니다."
        ),
        Document(
            page_content="한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다."
        ),
        Document(
            page_content="미국의 수도는 워싱턴이고, 재팬은 도교이며 코리아는 서울이다."
        ),
        Document(page_content="대한민국은 동아시에 위치한 나라이며, 분단국가이다."),
    ]

In [7]:
from langchain_openai import OpenAIEmbeddings


vector_store = Chroma.from_documents(get_sample_docs(), OpenAIEmbeddings())
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [8]:
query = "대한민국의 수도는?"

docs = retriever.invoke(query)

pretty_print_docs(docs)

Document 1:

많은 사람들이 대구를 대한민국의 수도로 착각하지만, 실제 수도는 아닙니다.
----------------------------------------------------------------------------------------------------
Document 2:

대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다.
----------------------------------------------------------------------------------------------------
Document 3:

한국의 수도는 서울이며, 세계적으로 유명한 도시입니다.
----------------------------------------------------------------------------------------------------
Document 4:

한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다.
----------------------------------------------------------------------------------------------------
Document 5:

서울은 대한민국의 수도로, 정치, 경제, 문화의 중심지입니다.
----------------------------------------------------------------------------------------------------
Document 6:

서울은 대한민국의 수도로서, 1948년부터 공식적으로 지정되었습니다.
----------------------------------------------------------------------------------------------------
Document 7:

미국의 수도는 워싱턴이고, 일본은 도쿄이며 북한은 평양이다.
--------------------------------

# bge-reranker
- 다국어 지원 BGE Reranker: [bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3)

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# 모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# 상위 3개의 문서 선택
compressor = CrossEncoderReranker(model=model, top_n=5)

# 문서 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("대한민국의 수도는?")

# 압축된 문서 출력
pretty_print_docs(compressed_docs)

Document 1:

한국의 수도는 서울이며, 세계적으로 유명한 도시입니다.
----------------------------------------------------------------------------------------------------
Document 2:

한국의 수도는 서울이며, 세계적으로 유명한 도시입니다.
----------------------------------------------------------------------------------------------------
Document 3:

대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다.
----------------------------------------------------------------------------------------------------
Document 4:

대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다.
----------------------------------------------------------------------------------------------------
Document 5:

한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다.


이제 기본 `retriever`를 `ContextualCompressionRetriever`로 감싸보겠습니다. `CrossEncoderReranker`는 `HuggingFaceCrossEncoder`를 사용하여 반환된 결과를 재정렬합니다.